In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from itertools import product
from math import sin, cos, asin, acos, radians
from pulp import LpProblem, LpMinimize, LpVariable, LpBinary, lpSum, LpStatus, value
import matplotlib.markers as mmarkers
import matplotlib.font_manager as fm
from matplotlib.colors import ListedColormap

## Data preparation

In [ ]:
df = pd.read_excel('../Db_Corretto.xlsx')
df = df.rename(columns = {'Latitudine' : 'lat', 'Longitudine' : 'lng'})

In [ ]:
def add_geocoordinates(df, lat='lat', lng='lng'):
    '''
    Add column 'geometry' with <shapely.geometry.point.Point> objects 
        built from latitude and longitude values in the input dataframe
    
    Args:
        - df: input dataframe
        - lat: name of the column containing the latitude (default: lat)
        - lng: name of the column containing the longitude (default: lng)
    Out:
        - df: same dataframe enriched with a geo-coordinate column
    '''
    assert pd.Series([lat, lng]).isin(df.columns).all(),\
        f'Cannot find columns '{lat}' and/or '{lng}' in the input dataframe.'
    return gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.lng, df.lat))

In [ ]:
df = add_geocoordinates(df)

In [ ]:
# Load shapefile of Italian regions and provinces
italy_reg = gpd.read_file('../Reg01012023_g_WGS84.shp')
italy_prov = gpd.read_file('../ProvCM01012023_g_WGS84.shp')

# Check and convert CRS if needed
if italy_reg.crs.to_string() != 'EPSG:4326':
    italy_reg = italy_reg.to_crs('EPSG:4326')
if italy_prov.crs.to_string() != 'EPSG:4326':
    italy_prov = italy_prov.to_crs('EPSG:4326')

# Filter the cluster provinces
mantova = italy_prov[italy_prov['DEN_PROV'] == 'Mantova']
pavia = italy_prov[italy_prov['DEN_PROV'] == 'Pavia']
milano = italy_prov[italy_prov['DEN_CM'] == 'Milano']

# Combine the geometries of Verona, Pavia, and Milano
cluster_boundary = gpd.GeoSeries(pd.concat([mantova.geometry, pavia.geometry, milano.geometry])).unary_union

# Generate grid of points within the cluster boundary
# Define grid parameters (adjust these as needed)
xmin, ymin, xmax, ymax = cluster_boundary.bounds
grid_size = 0.07  # Grid cell size in degrees

# Generate grid points
grid_points = []
for x, y in product(np.arange(xmin, xmax, grid_size), np.arange(ymin, ymax, grid_size)):
    point = Point(x, y)
    if cluster_boundary.contains(point):
        grid_points.append(point)

# Create a DataFrame with all grid points and their coordinates
grid_data = {
    'longitude': [point.x for point in grid_points],
    'latitude': [point.y for point in grid_points]
}
grid_point = pd.DataFrame(grid_data)

# Plot map
fig, ax = plt.subplots(figsize=(30, 30))
italy_prov.plot(ax=ax, color='lightgray', edgecolor='black')

# Plot customer locations
df.plot(ax=ax, marker='.', color='red', markersize=30, alpha=0.5, label='Customer')

# Plot grid points
ax.scatter(grid_point['longitude'], grid_point['latitude'], color='blue', alpha=0.5, label='Potential Warehouses', s=6)

ax.set_xticks([])
ax.set_yticks([])
ax.legend(facecolor='white', title='Location')
ax.set_title('Customer and potential warehouses')
plt.show()
grid_point.to_csv('grid_points_cluster.csv', index=False)

In [ ]:
# Import the warehouse points with locations
warehouses = pd.read_excel('warehouses.xlsx')
# Import the provincies dataset
provinces = pd.read_excel('../province.xlsx')
provinces = provinces[['Official Name Provincia/Città metropolitana', 'Official Name Regione']].rename(columns={'Official Name Regione': 'Regione'})
# Import the municipality dataset
municipalities = gpd.read_file('../Com01012023_g_WGS84.shp')
municipalities = municipalities[['COD_REG', 'COD_PROV', 'COMUNE', 'Shape_Leng', 'geometry']]
if municipalities.crs.to_string() != 'EPSG:4326':
    municipalities = municipalities.to_crs('EPSG:4326')

In [ ]:
# Fill NAs in 'Milano' province with 'Milano'
warehouses['Località'] = np.where((warehouses['Località'].isna()) & (warehouses['Provincia'] == 'Milano'), 'Milano', warehouses['Località'])

In [ ]:
# Add the centroids to the warehouses dataframe
centroids = pd.read_excel('centroids.xlsx')
warehouses = pd.concat([warehouses, centroids], ignore_index = True)

In [ ]:
# Drop the unnecessary columns
warehouses.drop(columns=['CAP'], inplace=True)

In [ ]:
# Join 'warehouses' and 'provinces' together
warehouses = warehouses.join(provinces.set_index('Official Name Provincia/Città metropolitana'), on = 'Provincia')

## Logistic companies data

In [ ]:
# Import the logistic companies dataset
logistic = pd.read_csv('numero_imprese.csv')
# Filter based on ANNO == 2020
logistic = logistic[logistic['ANNO'] == 2020]
logistic = logistic[['Descrizione_Comune', 'H']]

In [ ]:
# Merge the shapefile data with logistic data
municipalities = municipalities.merge(logistic, left_on='COMUNE', right_on='Descrizione_Comune')

In [ ]:
lombardia = italy_reg[italy_reg['DEN_REG'] == 'Lombardia']

# Define breaks, labels and colours
breaks = [-1, 0, 5, 10, 50, 100, 300, float('inf')]
labels = ['0', '1-5', '6-10', '11-50', '51-100', '101-300', '301+']
municipalities['H_cuts'] = pd.cut(municipalities['H'], bins=breaks, labels=labels, right=False)
colors = {'0': 'white', '1-5': '#ffffcc', '6-10': '#ffeda0', '11-50': '#fec44f', 
          '51-100': '#fd8d3c', '101-300': '#fc4e2a', '301+': '#e31a1c'}
cmap = ListedColormap(['white', '#ffffcc', '#ffeda0', '#fec44f', '#fd8d3c', '#fc4e2a', '#e31a1c'])

# Plot the map
fig, ax = plt.subplots(figsize=(14, 10), facecolor='white')
lombardia.boundary.plot(ax=ax, linewidth=0.5, edgecolor='gray')

# Plot municipalities within Lombardia with color-coded logistic companies
municipalities[municipalities['COD_REG'] == 3].plot(ax=ax, column='H_cuts', cmap=cmap, legend=True, linewidth=0.5, edgecolor='black', legend_kwds={'loc': 'upper left'})

ax.set_title('Distribution of Logistic Companies by Municipality', fontsize=20, fontweight='bold', pad=20,
             fontdict={'fontsize': 20, 'fontweight': 'bold', 'family': 'serif', 'style': 'italic'})
ax.set_xlabel('Longitude', fontsize=14, fontweight='normal', family='serif')
ax.set_ylabel('Latitude', fontsize=14, fontweight='normal', family='serif')

legend = ax.get_legend()
legend.set_title('Number of Companies')
legend.set_bbox_to_anchor((0.8, 0.8))
legend.set_frame_on(False)

ax.tick_params(axis='both', which='both', length=0)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.savefig('Logistic_Companies.png', transparent=False)
plt.tight_layout()
plt.show()

In [ ]:
# Merge the logistic companies data into 'warehouses'
warehouses = warehouses.merge(logistic[['Descrizione_Comune', 'H']], left_on='Località', right_on='Descrizione_Comune', how='left')
warehouses.drop(columns=['Descrizione_Comune'], inplace=True)

In [ ]:
# Drop all the warehouses with at least a NaN
warehouses.dropna(inplace=True)
# Add the 'warehouse_id' column
warehouses.insert(0, 'warehouse_id', [f'Warehouse {i+1}' for i in range(warehouses.shape[0])])

## Workforce

In [ ]:
# Import the population dataset
population = pd.read_excel('pop.xlsx')

In [ ]:
# Drop the last row
population = population[:-1]
# Drop the unnecessary columns
population = population.drop(columns=['Codice comune', 'Totale maschi', 'Totale femmine'])
# Filter rows where 'Età' is between 18 and 60
population = population[(population['Età'] >= 18) & (population['Età'] <= 60)]

In [ ]:
# Group by 'Comune' and sum
population_sum = population.groupby('Comune')['Totale'].sum().reset_index()
population_sum.rename(columns={'Totale': 'workers'}, inplace=True)

In [ ]:
# Merge the shapefile data with workforce data
municipalities = municipalities.merge(population_sum, left_on='COMUNE', right_on='Comune')

In [ ]:
# Drop the unnecessary columns
municipalities = municipalities.drop(columns=['Descrizione_Comune', 'Comune'])

In [ ]:
# Define breaks, labels and colours
breaks = [0, 500, 1000, 2500, 5000, 10000, 30000, float('inf')]
labels = ['0-500', '501-1000', '1001-2500', '2501-5000', '5001-10000', '10001-30000', '30000+']
municipalities['worker_cuts'] = pd.cut(municipalities['workers'], bins=breaks, labels=labels, right=False)
colors = {'0-500': 'white', '501-1000': '#ffffcc', '1001-2500': '#ffeda0', '2501-5000': '#fec44f', 
          '5001-10000': '#fd8d3c', '10001-30000': '#fc4e2a', '30000+': '#e31a1c'}
cmap = ListedColormap(['white', '#ffffcc', '#ffeda0', '#fec44f', '#fd8d3c', '#fc4e2a', '#e31a1c'])

# Plot the map
fig, ax = plt.subplots(figsize=(14, 10), facecolor='white')
lombardia.boundary.plot(ax=ax, linewidth=0.5, edgecolor='gray')

# Plot municipalities within Lombardia with color-coded cuts
municipalities[municipalities['COD_REG'] == 3].plot(ax=ax, column='worker_cuts', cmap=cmap, legend=True, linewidth=0.5, edgecolor='black', legend_kwds={'loc': 'upper left'})


ax.set_title('Distribution of the Workforce by Municipality', fontsize=20, fontweight='bold', pad=20,
             fontdict={'fontsize': 20, 'fontweight': 'bold', 'family': 'serif', 'style': 'italic'})
ax.set_xlabel('Longitude', fontsize=14, fontweight='normal', family='serif')
ax.set_ylabel('Latitude', fontsize=14, fontweight='normal', family='serif')

legend = ax.get_legend()
legend.set_title('Workers')
legend.set_bbox_to_anchor((0.8, 0.8))
legend.set_frame_on(False)

ax.tick_params(axis='both', which='both', length=0)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.savefig('Workforce.png', transparent=False)
plt.tight_layout()
plt.show()

In [ ]:
# Merge the workforce data into 'warehouses'
warehouses = warehouses.merge(population_sum[['Comune', 'workers']], left_on='Località', right_on='Comune', how='left')
warehouses.drop(columns=['Comune'], inplace=True)

In [ ]:
# Drop all the warehouses with at least a NaN
warehouses.dropna(inplace=True)

## Fixed costs

In [ ]:
# Create a dataset with the €/mq for every province
cost_mq = {
    'Province': ['Bergamo', 'Brescia', 'Como', 'Cremona', 'Lecco', 'Lodi', 'Mantova', 'Milano', 'Monza e della Brianza', 'Pavia', 'Sondrio', 'Varese'],
    'euro_per_sqm': [1591, 2166, 2098, 1234, 1647, 1398, 1162, 3815, 2125, 1196, 1662, 1579]
}
cost_mq = pd.DataFrame(cost_mq)
# Source: https://www.immobiliare.it/mercato-immobiliare/lombardia/

In [ ]:
# Define the dimension of the warehouses
dim = 1500
# Source: https://www.wh1.com/warehouse-square-footage-tips/

In [ ]:
# Calculate the fixed cost for every warehouse
warehouses = warehouses.merge(cost_mq, left_on='Provincia', right_on='Province', how='left')
warehouses['fixed_costs'] = warehouses['euro_per_sqm'] * dim / 25
warehouses.drop('Province', axis=1, inplace=True)

# Source: https://quickmastro.it/download/manuali/TabellaCoefficientiAmmortamenti/TABELLA%20DEI%20COEFFICIENTI%20DI%20AMMORTAMENTO.html?GruppoXVIIIIndustriedeitrasporti.html

## Variable costs

In [ ]:
def haversine_distance(lat1, lon1, lat2, lon2):
    '''
    Calculate distance between two locations given latitude and longitude.

    Args:
       - lat1: latitude of the first location
       - lon1: longitude of the first location
       - lat2: latitude of the second location
       - lon2: longitude of the second location
    Out:
       - Distance in Km
    
    '''
    return 6371.01 *\
            acos(sin(radians(lat1))*sin(radians(lat2)) +\
            cos(radians(lat1))*cos(radians(lat2))*cos(radians(lon1)-radians(lon2)))

In [ ]:
def traveling_cost(distance_in_km):
    '''
    Return traveling cost in euros given a distance in Km.

    Args:
      - distance_in_km: travel distance in Km
    Out:
      - cost of the trip in euros
    '''
    return 1.203 * distance_in_km

# Source: https://www.truck24.it/quanto-costa-un-trasporto-al-km-aggiornati-i-dati-a-gennaio-2024/

In [ ]:
# Calculate the distances between all warehouses and customers
transport_costs_dict = {}

for i in range(warehouses.shape[0]):
    warehouse_transport_costs_dict = {}

    warehouse_id = warehouses.iloc[i]['warehouse_id']
    print(f'Processing warehouse_id: {warehouse_id} ({i+1}/{warehouses.shape[0]})')

    for j in range(df.shape[0]):
        if warehouses.iloc[i]['Località'] == df.iloc[j]['Località']:
            d = 0
        else:
            d = haversine_distance(
                warehouses.iloc[i]['latitude'], 
                warehouses.iloc[i]['longitude'], 
                df.iloc[j]['lat'], 
                df.iloc[j]['lng']
            )

        unique_key = f'{df.iloc[j]['Cliente']}_{j}'
        warehouse_transport_costs_dict[unique_key] = traveling_cost(d)

    transport_costs_dict[warehouse_id] = warehouse_transport_costs_dict

print(f'Total keys in transport_costs_dict: {len(transport_costs_dict.keys())}')

In [ ]:
print(f'Number of keys in transport_costs_dict: {len(transport_costs_dict.keys())}')

total_entries = sum(len(v) for v in transport_costs_dict.values())
print(f'Total number of entries across all warehouses: {total_entries}')

## Optimization

In [ ]:
# Define the linear problem
lp_problem = LpProblem('Warehouse Optimization', LpMinimize)

# Variables

# Unique_key for each customer
unique_keys = [f"{df.iloc[j]['Cliente']}_{j}" for j in range(len(df))]

# y_j: Binary variable indicating if warehouse j is opened
created_facility = LpVariable.dicts(
    'Create_facility', warehouses['warehouse_id'], cat=LpBinary)

# served_customer variables using unique_key
served_customer = LpVariable.dicts(
    'Link', [(i, j) for i in unique_keys for j in warehouses['warehouse_id']], lowBound=0)

# Contraints

# Constraint: Every customer must be linked to exactly one warehouse
for unique_key in unique_keys:
    lp_problem += lpSum(served_customer[(unique_key, k)] for k in warehouses['warehouse_id']) == 1, f'Customer_{unique_key}_Assignment'

# Constraint: Number of warehouses opened must be between 1 and 3
lp_problem += lpSum(created_facility[j] for j in warehouses['warehouse_id']) >= 1
lp_problem += lpSum(created_facility[j] for j in warehouses['warehouse_id']) <= 3

# Constraint: Link variable should only be active if warehouse is created
for unique_key in unique_keys:
    for j in warehouses['warehouse_id']:
        lp_problem += served_customer[(unique_key, j)] <= created_facility[j], f'Serving_Constraint_{unique_key}_{j}'

In [ ]:
# Optimization function

# Fixed costs
fixed_costs = lpSum(warehouses.loc[j, 'fixed_costs'] * created_facility['Warehouse ' + str(j+1)] for j in range(len(warehouses)))
objective = fixed_costs

# Variable costs
variable_costs = lpSum(transport_costs_dict[warehouse_id][unique_key] * served_customer[(unique_key, warehouse_id)] 
                       for warehouse_id in warehouses['warehouse_id'] 
                       for unique_key in unique_keys 
                       if unique_key in transport_costs_dict[warehouse_id])
objective += variable_costs

# Logistic companies
weight_H = -50000
warehouses['H_normal'] = (warehouses['H'] - min(warehouses['H'])) / (max(warehouses['H']) - min(warehouses['H']))

logistic_companies_modifier = lpSum(
    warehouses.loc[j, 'H_normal'] * weight_H * created_facility[warehouse_id]
    for j, warehouse_id in enumerate(warehouses['warehouse_id'])
)
objective += logistic_companies_modifier

# Workforce
weight_workers = -20000
warehouses['workers_normal'] = (warehouses['workers'] - min(warehouses['workers'])) / (max(warehouses['workers']) - min(warehouses['workers']))

workforce_modifier = lpSum(
    warehouses.loc[j, 'workers_normal'] * weight_workers * created_facility[warehouse_id]
    for j, warehouse_id in enumerate(warehouses['warehouse_id'])
)
objective += workforce_modifier

lp_problem += objective 

In [ ]:
print(f'Number of variables: {len(lp_problem.variables())}')
print(f'Number of constraints: {len(lp_problem.constraints)}')

In [ ]:
lp_problem.solve()

print('Status:', LpStatus[lp_problem.status])
print('Objective Value (Total Cost):', value(lp_problem.objective))

opened_warehouses = [w for w in warehouses['warehouse_id'] if created_facility[w].varValue == 1]
print('\nOpened Warehouses:', opened_warehouses)

In [ ]:
# Identify customers served by each warehouse
customers_served_by_warehouse = {w: [] for w in warehouses['warehouse_id']}

for warehouse_id in warehouses['warehouse_id']:
    if created_facility[warehouse_id].varValue == 1:
        for unique_key in unique_keys:
            if served_customer[(unique_key, warehouse_id)].varValue > 0:
                customers_served_by_warehouse[warehouse_id].append(unique_key)

# Print customers not served by any warehouse
not_served_customers = []
for unique_key in unique_keys:
    served = False
    for warehouse_id in warehouses['warehouse_id']:
        if created_facility[warehouse_id].varValue == 1 and served_customer[(unique_key, warehouse_id)].varValue > 0:
            served = True
            break
    if not served:
        not_served_customers.append(unique_key)

print('\nCustomers Not Served by Any Warehouse:')
if len(not_served_customers) == 0:
    print('All customers are served')

for unique_key in not_served_customers:
    print(f'- {unique_key}')

# Count and display the number of customers served by each warehouse
print('\nNumber of Customers Served by Each Warehouse:')
for warehouse_id, customers in customers_served_by_warehouse.items():
    print(f'Warehouse {warehouse_id}: {len(customers)} customers')


## Visualization

In [ ]:
# Create dataframe column to store whether to build the warehouse or not 
warehouses['build_warehouse'] = ''

# Assign Yes/No to the dataframe column based on the optimization binary variable
for i in warehouses['warehouse_id']:
    if created_facility[i].varValue == 1:
        print('Build site at: ', i)
        warehouses.loc[warehouses['warehouse_id'] == i, 'build_warehouse'] = 'Yes'
    else:
        warehouses.loc[warehouses['warehouse_id'] == i, 'build_warehouse'] = 'No'
colors = ['#990000', '#0059b3']

warehouses.build_warehouse.value_counts().plot.barh(
  title='Warehouse sites to be established', xlabel='Number of sites', color=colors, ylabel='Establish', figsize=(7,6)) 

for i, v in enumerate(warehouses.build_warehouse.value_counts()):
    plt.text(v, i, ' '+str(round(v,3)), color=colors[i], va='center', fontweight='bold')

In [ ]:
# Save the final df
warehouses.to_csv('final_warehouses.csv', index=False)

In [ ]:
# Save the served customers dict
served_customers_list = []

for warehouse_id, unique_ids in customers_served_by_warehouse.items():
    if warehouses.loc[warehouses['warehouse_id'] == warehouse_id, 'build_warehouse'].values[0] == 'Yes':
        for unique_id in unique_ids:
            served_customers_list.append({'unique_id': unique_id, 'warehouse_id': warehouse_id})

served_customers = pd.DataFrame(served_customers_list)
served_customers.to_csv('final_served_customers.csv', index=False)

In [ ]:
warehouses = pd.read_csv('final_warehouses.csv')
clients = pd.read_csv('final_served_customers.csv')

In [ ]:
grid_point = pd.concat([grid_point, centroids], ignore_index=True)
grid_point.drop(columns=['Località', 'Provincia', 'CAP'])

In [ ]:
# Plot the map
fig, ax = plt.subplots(figsize=(14, 10), facecolor='white')
xmin, xmax = lombardia.bounds['minx'].min(), lombardia.bounds['maxx'].max()
ymin, ymax = lombardia.bounds['miny'].min(), lombardia.bounds['maxy'].max()
lombardia_prov = italy_prov[italy_prov.COD_REG == 3]
lombardia_prov.plot(ax=ax, color='#f0f0f0', edgecolor='black', linewidth=0.2, zorder=1)

# Plot grid points
ax.scatter(grid_point.iloc[:-3]['longitude'], grid_point.iloc[:-3]['latitude'], 
           color='red', alpha=0.8, label='Discarded warehouses', s=10, 
           marker='x', zorder=2)

# Filter warehouses to only those where 'build_warehouse' is 'Yes'
built_warehouses = warehouses[warehouses['build_warehouse'] == 'Yes']

# Plot built warehouses
ax.scatter(built_warehouses['longitude'], built_warehouses['latitude'], 
           color='blue', s=50, alpha=0.8, label='Built warehouses',
           marker=mmarkers.MarkerStyle('v', fillstyle='full'), zorder=4,
           edgecolors='black', linewidth=1.5)

# Plot the initial centroids
ax.scatter(grid_point.iloc[-3:]['longitude'], grid_point.iloc[-3:]['latitude'], 
           color='green', alpha=0.8, label='Centroids', s=50, 
           marker='o', zorder=3)

ax.set_xlim(xmin-0.1, xmax+0.1)
ax.set_ylim(ymin-0.1, ymax+0.1)

ax.tick_params(axis='both', which='both', length=0)

ax.set_xlabel('Longitude', fontsize=14, fontweight='normal', family='serif')
ax.set_ylabel('Latitude', fontsize=14, fontweight='normal', family='serif')
ax.set_title('Possible Warehouse Locations', fontsize=20, fontweight='bold', pad=20,
             fontdict={'fontsize': 20, 'fontweight': 'bold', 'family': 'serif', 'style': 'italic'})

legend = ax.legend(loc='upper left', fontsize=12, frameon=False, shadow=True, fancybox=True,
                   prop={'family': 'serif', 'weight': 'normal'})
legend.get_frame().set_facecolor('white')
legend.get_frame().set_edgecolor('#cccccc')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.savefig('Warehouse_Location.png', transparent=False)
plt.tight_layout()
plt.show()

In [ ]:
merged_data = pd.merge(df, clients, on='unique_id')
# Define color map
warehouse_ids = merged_data['warehouse_id'].unique()
num_warehouses = len(warehouse_ids)
color_map = plt.cm.get_cmap('bwr', num_warehouses)

# Plot the map
fig, ax = plt.subplots(figsize=(14, 10), facecolor='white')
xmin, xmax = 6, 15
ymin, ymax = 42, 48 
italy_reg.plot(ax=ax, color='#f0f0f0', edgecolor='black', linewidth = 0.1)
lombardia[lombardia.cx[xmin:xmax, ymin:ymax].notna()].plot(ax=ax, color='none', edgecolor='#505050', linewidth=0.4, zorder=3)
lombardia_prov[lombardia_prov.cx[xmin:xmax, ymin:ymax].notna()].plot(ax=ax, color='none', edgecolor='#a0a0a0', linewidth=0.2, zorder=2)

# Plot customer locations for each warehouse_id
for i, wh_id in enumerate(warehouse_ids):
    wh_data = merged_data[merged_data['warehouse_id'] == wh_id]
    ax.scatter(wh_data['lng'], wh_data['lat'], 
               c=[color_map(i)] * len(wh_data),
               s=0.5, alpha=0.6, 
               label=f'Customers ({wh_id})',
               zorder=4)
# Plot built warehouses
for i, wh_id in enumerate(warehouse_ids):
    wh_data = built_warehouses[built_warehouses['warehouse_id'] == wh_id]
    ax.scatter(wh_data['longitude'], wh_data['latitude'], 
               color=color_map(i), s=150, alpha=0.8, label=wh_id, 
               marker=mmarkers.MarkerStyle('v', fillstyle='full'), zorder=5,
               edgecolors='black', linewidth=1.5) 


ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

ax.grid(False)
ax.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False)

ax.set_xlabel('Longitude', fontdict={'fontsize': 14, 'fontweight': 'normal', 'family': 'serif'})
ax.set_ylabel('Latitude', fontdict={'fontsize': 14, 'fontweight': 'normal', 'family': 'serif'})
ax.set_title('Warehouse Distribution and Customer Locations', fontsize=20, fontweight='bold', pad=20,
             fontdict={'fontsize': 20, 'fontweight': 'bold', 'family': 'serif', 'style': 'italic'})

legend = ax.legend(loc='upper left', fontsize=12, frameon=False, shadow=True, fancybox=True, 
                   prop={'family': 'serif', 'weight': 'normal'})
legend.get_frame().set_facecolor('white')
legend.get_frame().set_edgecolor('#cccccc')

ax.spines['top'].set_visible(False)    
ax.spines['right'].set_visible(False)  
ax.spines['bottom'].set_visible(False) 
ax.spines['left'].set_visible(False)   

plt.savefig('Warehouse_Distribution.png', facecolor='white', transparent=False)
plt.tight_layout()
plt.show()